In [1]:
import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
from sklearn.model_selection import train_test_split
from tensorflow_addons.metrics.r_square import RSquare
import tensorflow_addons as tfa
from sklearn.compose import make_column_transformer
from sklearn.preprocessing import MinMaxScaler, OneHotEncoder

2022-07-02 23:10:01.763501: I tensorflow/core/util/util.cc:169] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.


In [ ]:
tf.random.set_seed(
    42
)

In [2]:
# Load data
data = pd.read_csv('dataset/real/insurance.csv')
data.drop(columns=['Unnamed: 0'],inplace=True)

In [3]:
# Turn all categories into numbers
insurance_one_hot = pd.get_dummies(data)
insurance_one_hot.head() # view the converted columns
# Create X & y values
X = insurance_one_hot.drop("charges", axis=1)
y = insurance_one_hot["charges"]

In [4]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(
    X,
    y,
    test_size = 0.2,
    random_state=42
)

In [5]:
# Modeling
model = tf.keras.Sequential([
    tf.keras.layers.Dense(100),
    tf.keras.layers.Dense(10),
    tf.keras.layers.Dense(5),
    tf.keras.layers.Dense(1),
])

# Compile
model.compile(
    loss=tf.keras.losses.mae,
    optimizer = tf.keras.optimizers.Adam(),
    metrics=['mae']
)

# Fit the model
history = model.fit(X_train.values, y_train.values, epochs= 100, verbose=1,  validation_data=(X_test.values,y_test.values))

Epoch 1/100


2022-07-02 23:10:03.914636: E tensorflow/stream_executor/cuda/cuda_driver.cc:271] failed call to cuInit: CUDA_ERROR_UNKNOWN: unknown error
2022-07-02 23:10:03.914664: I tensorflow/stream_executor/cuda/cuda_diagnostics.cc:169] retrieving CUDA diagnostic information for host: tari-Nitro-AN515-56
2022-07-02 23:10:03.914669: I tensorflow/stream_executor/cuda/cuda_diagnostics.cc:176] hostname: tari-Nitro-AN515-56
2022-07-02 23:10:03.914769: I tensorflow/stream_executor/cuda/cuda_diagnostics.cc:200] libcuda reported version is: 470.129.6
2022-07-02 23:10:03.914784: I tensorflow/stream_executor/cuda/cuda_diagnostics.cc:204] kernel reported version is: 470.129.6
2022-07-02 23:10:03.914792: I tensorflow/stream_executor/cuda/cuda_diagnostics.cc:310] kernel version seems to match DSO: 470.129.6
2022-07-02 23:10:03.915241: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions i

34/34 [==============================] - 0s 4ms/step - loss: 13307.2598 - mae: 13307.2598 - val_loss: 12876.0889 - val_mae: 12876.0889
Epoch 2/100
34/34 [==============================] - 0s 2ms/step - loss: 13159.2822 - mae: 13159.2822 - val_loss: 12634.0244 - val_mae: 12634.0244
Epoch 3/100
34/34 [==============================] - 0s 2ms/step - loss: 12736.3545 - mae: 12736.3545 - val_loss: 11940.2705 - val_mae: 11940.2705
Epoch 4/100
34/34 [==============================] - 0s 2ms/step - loss: 11610.2754 - mae: 11610.2754 - val_loss: 10271.4180 - val_mae: 10271.4180
Epoch 5/100
34/34 [==============================] - 0s 2ms/step - loss: 9491.3643 - mae: 9491.3643 - val_loss: 8098.1548 - val_mae: 8098.1548
Epoch 6/100
34/34 [==============================] - 0s 2ms/step - loss: 7651.2603 - mae: 7651.2603 - val_loss: 7614.8281 - val_mae: 7614.8281
Epoch 7/100
34/34 [==============================] - 0s 2ms/step - loss: 7447.6650 - mae: 7447.6650 - val_loss: 7587.2476 - val_mae: 7587.

In [7]:
# Evaulate 3rd model by metrics
model_loss, model_mae = model.evaluate(X_test, y_test)

9/9 [==============================] - 0s 802us/step - loss: 3587.9299 - mae: 3587.9299


In [ ]:
%load_ext tensorboard

In [8]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense (Dense)               (None, 100)               1200      
                                                                 
 dense_1 (Dense)             (None, 10)                1010      
                                                                 
 dense_2 (Dense)             (None, 5)                 55        
                                                                 
 dense_3 (Dense)             (None, 1)                 6         
                                                                 
Total params: 2,271
Trainable params: 2,271
Non-trainable params: 0
_________________________________________________________________


## Tuning

In [9]:
import optuna
from optuna.integration import TFKerasPruningCallback
from optuna.trial import TrialState
from optuna.visualization import plot_intermediate_values
from optuna.visualization import plot_optimization_history
from optuna.visualization import plot_param_importances
from optuna.visualization import plot_contour

In [10]:
def create_model(trial):
    # Modeling
    num_node_layer1 = trial.suggest_int("layer1", 50, 100)
    num_node_layer2 = trial.suggest_int("layer2", 10, 50)
    num_node_layer3 = trial.suggest_int("layer3", 1, 10)
    lr = trial.suggest_float('lr',5e-4,5e-3)
    beta1 = trial.suggest_float('b1',0.85,0.95)
    beta2 = trial.suggest_float('b2',0.99,0.999)
    model = tf.keras.Sequential([
        tf.keras.layers.Dense(num_node_layer1),
        tf.keras.layers.Dense(num_node_layer2),
        tf.keras.layers.Dense(num_node_layer3),
        tf.keras.layers.Dense(1),
    ])

    # Compile
    model.compile(
        loss=tf.keras.losses.mae,
        optimizer = tf.keras.optimizers.Adam(
            learning_rate=lr,
            beta_1= beta1,
            beta_2= beta2
        ),
        metrics=['mae']
    )
    return model


In [12]:
EPOCHS = 100 # number of epocs per trial
def objective(trial):
    model = create_model(trial)
    # Fit the model on the training data.
    model.fit(
        X_train.values,
        y_train.values,
        epochs=EPOCHS,
        validation_data=(X_test.values, y_test.values),
        verbose=0,
    )

    # Evaluate the model accuracy on the validation set.
    score = model.evaluate(X_test, y_test, verbose=0)
    return score[1]

In [13]:
study = optuna.create_study(direction="minimize", sampler=optuna.samplers.TPESampler(), pruner=optuna.pruners.HyperbandPruner())
study.optimize(objective, n_trials=50)
pruned_trials = study.get_trials(deepcopy=False, states=[TrialState.PRUNED])
complete_trials = study.get_trials(deepcopy=False, states=[TrialState.COMPLETE])

[I 2022-07-02 23:11:48,551] A new study created in memory with name: no-name-607838d1-8809-4dbc-9f96-dba94a03a470
[I 2022-07-02 23:11:53,493] Trial 0 finished with value: 3925.552734375 and parameters: {'layer1': 79, 'layer2': 32, 'layer3': 6, 'lr': 0.0005786558625862741, 'b1': 0.8965436603529848, 'b2': 0.991540751674065}. Best is trial 0 with value: 3925.552734375.
[I 2022-07-02 23:11:58,384] Trial 1 finished with value: 3160.989990234375 and parameters: {'layer1': 58, 'layer2': 32, 'layer3': 3, 'lr': 0.0029508522781749587, 'b1': 0.9235362122717298, 'b2': 0.9988318801916549}. Best is trial 1 with value: 3160.989990234375.
[I 2022-07-02 23:12:03,061] Trial 2 finished with value: 3159.830810546875 and parameters: {'layer1': 72, 'layer2': 46, 'layer3': 4, 'lr': 0.0030675664250030344, 'b1': 0.8970737529032222, 'b2': 0.9937217356850564}. Best is trial 2 with value: 3159.830810546875.
[I 2022-07-02 23:12:07,718] Trial 3 finished with value: 3173.924560546875 and parameters: {'layer1': 51, '

In [ ]:
plot_optimization_history(study)